In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import textblob
import collections
import json
import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
training_data = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
training_data.head()

In [ ]:
training_data['dataset_label'].unique()

I am goint to use *Pointwise Mutual Information* to try to extract key phrases from the documents, and then match the key phrases to the dataset names.
Pointwise Mutual Information will be calculated on pairs of successive words from within a sentence, after removal of stopwords.

In [ ]:
individual_counts = collections.defaultdict(float)
pair_counts = collections.defaultdict(float)
n_words = 0.0
n_sentences = 0.0
stopwords = nltk.corpus.stopwords.words('english')
for filename in tqdm.tqdm(os.listdir('../input/coleridgeinitiative-show-us-the-data/train')):
    with open('/'.join(('../input/coleridgeinitiative-show-us-the-data/train',filename))) as data:
        article = json.load(data)
        text = '\n'.join(['\n'.join((section['section_title'],section['text']))
                          for section in article])
        sentences = textblob.TextBlob(text).sentences
        n_sentences += len(sentences)
        for sentence in sentences:
            words = [str(word) for word in sentence.lower().words
                            if str(word) not in stopwords]
            if len(words)>0:
                n_words += len(words)
                for (i,word) in enumerate(words[:-1]):
                    individual_counts[word] +=1.0
                    pair_counts[(word,words[i+1])] += 1.0
                individual_counts[words[-1]]+=1.0
            else:
                n_sentences -= 1

pmi = pd.Series({(word0,word1):(n * ((n_words - n_sentences)**2.0))/(n_words * individual_counts[word0] * individual_counts[word1])
                 for ((word0,word1),n) in tqdm.tqdm(pair_counts.items())
                if n > 1}).apply(np.log2)

pmi.quantile(np.linspace(0.0,1.0,101)).plot.line()
            
        

In [ ]:
pmi[pmi>=3]

Now let us find some candidate sequences from each document. These are sequences of words from within a sentence where (not counting stopwords) each pair of successive words has a Pointwise Mutual Information > 3 bits.

In [ ]:
def candidates(document):
    text = '\n'.join(['\n'.join((section['section_title'],section['text']))
                     for section in article])
    for sentence in textblob.TextBlob(text).sentences:
        prev_word = None
        prev_index = None
        span = []
        words = [str(word) for word in sentence.lower().words]
        for (i,word) in enumerate(words):
            if word not in stopwords:
                score = pmi.get((prev_word,word),0)
                if score >3:
                    span.append(prev_word)
                elif len(span) > 0:
                    span.append(prev_word)
                    yield (span,' '.join(words[prev_index:i]))
                    span = []
                    prev_index = i
                else:
                    prev_index = i
                prev_word = word
        if len(span) > 0:
            span.append(prev_word)
            yield  (span,' '.join(words[prev_index:]))
    

In [ ]:
with open('../input/coleridgeinitiative-show-us-the-data/train/0007f880-0a9b-492d-9a58-76eb0b0e0bd7.json') as doc:
    for (span,string) in candidates(json.load(doc)):
        print(span)
        print(string)

In [ ]:
class LabelFilter(object):
    
    def __init__(self,data):
        self.labels = {tuple([word for word in label.split()
                        if word not in stopwords])
                       for label in data['cleaned_label'].unique()}
        
    def match(self,candidate,label):
        word = label[0]
        result = word in candidate
        if result:
            n = candidate.index(word)
            result = tuple(candidate[n:n+len(label)]) == label
        return result
    
    def __call__(self,candidate):
        return any((self.match(candidate,label)
                   for label in self.labels))     
    
label_filter = LabelFilter(training_data)

In [ ]:
with open('../input/coleridgeinitiative-show-us-the-data/train/0007f880-0a9b-492d-9a58-76eb0b0e0bd7.json') as doc:
    print([string for (span,string) in candidates(doc)
          if label_filter(string)])